In [103]:
#importing the important libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [104]:
#reading the data
df = pd.read_csv("model_data.csv", sep=";")
df.head()

,product_tier,make_name,price,first_zip_digit,first_registration_year,search_views,detail_views,stock_days,ctr,created_month,deleted_month
0,Basic,Mitsubishi,16750,5,2013,3091,123,31,0.037803,7,8
1,Basic,Mercedes-Benz,35950,4,2015,3283,223,52,0.067926,8,10
2,Basic,Mercedes-Benz,11950,3,1998,3247,265,51,0.081614,7,9
3,Basic,Ford,1750,6,2003,1856,26,101,0.014009,7,10
4,Basic,Mercedes-Benz,26500,3,2014,490,20,11,0.040816,8,9


To get the data ready for modelling we need to:

- Drop the "search_views" column to avoid multi-collinearity.

- Get numerical values for the categorical data (one hot encoding).

- Get a scaled copy of the data.

- Split the dataset into training and testing set and seperate the features from the target value. 

In [105]:
#dropping the search views and the duplicates
df.drop("search_views", axis=1, inplace=True)

df.drop_duplicates(inplace=True)

In [106]:
df.duplicated().sum()

0

In [107]:
#splitting features and target values
X = df.iloc[:, 1:]

y = df.iloc[:, 0]

In [108]:
X.head()

,make_name,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month
0,Mitsubishi,16750,5,2013,123,31,0.037803,7,8
1,Mercedes-Benz,35950,4,2015,223,52,0.067926,8,10
2,Mercedes-Benz,11950,3,1998,265,51,0.081614,7,9
3,Ford,1750,6,2003,26,101,0.014009,7,10
4,Mercedes-Benz,26500,3,2014,20,11,0.040816,8,9


In [109]:
y.head()

0    Basic
1    Basic
2    Basic
3    Basic
4    Basic
Name: product_tier, dtype: object

In [110]:
X.make_name.nunique()

91

- The column "make_name" has so many categories that if we on hot encoded them, we shall increase the dimensionality drastically.

- I will choose only the top 20 classes and one hot encode them, and give 0 for the rest of the classes.

In [111]:
#finding the top 20 classes
top20 = X.make_name.value_counts(ascending=False).head(20).index

In [112]:
#assigning the other classes to the value "n"
X.make_name = X.make_name.apply(lambda x: x.replace(x,"n") if x not in top20 else x)

In [113]:
X.make_name.value_counts(ascending=False)

Volkswagen       9417
n                7132
Renault          6929
Peugeot          5444
Opel             5215
Ford             5196
Mercedes-Benz    4757
BMW              4551
Volvo            3992
Toyota           3924
Audi             3311
Citroen          3017
Fiat             2586
Nissan           2015
Kia              1931
Hyundai          1802
SEAT             1772
Skoda            1515
Suzuki           1312
MINI             1234
Mazda            1198
Name: make_name, dtype: int64

In [114]:
X.head()

,make_name,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month
0,n,16750,5,2013,123,31,0.037803,7,8
1,Mercedes-Benz,35950,4,2015,223,52,0.067926,8,10
2,Mercedes-Benz,11950,3,1998,265,51,0.081614,7,9
3,Ford,1750,6,2003,26,101,0.014009,7,10
4,Mercedes-Benz,26500,3,2014,20,11,0.040816,8,9


In [115]:
#getting dummy variables for the categorical values
X = pd.get_dummies(X, drop_first=True)

In [116]:
X.head()

,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month,make_name_BMW,make_name_Citroen,...,make_name_Opel,make_name_Peugeot,make_name_Renault,make_name_SEAT,make_name_Skoda,make_name_Suzuki,make_name_Toyota,make_name_Volkswagen,make_name_Volvo,make_name_n
0,16750,5,2013,123,31,0.037803,7,8,0,0,...,0,0,0,0,0,0,0,0,0,1
1,35950,4,2015,223,52,0.067926,8,10,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11950,3,1998,265,51,0.081614,7,9,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1750,6,2003,26,101,0.014009,7,10,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26500,3,2014,20,11,0.040816,8,9,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
#removing the make_name part from the column names
X.columns = X.columns.str.replace('make_name_', '')

In [118]:
X.columns

Index(['price', 'first_zip_digit', 'first_registration_year', 'detail_views',
       'stock_days', 'ctr', 'created_month', 'deleted_month', 'BMW', 'Citroen',
       'Fiat', 'Ford', 'Hyundai', 'Kia', 'MINI', 'Mazda', 'Mercedes-Benz',
       'Nissan', 'Opel', 'Peugeot', 'Renault', 'SEAT', 'Skoda', 'Suzuki',
       'Toyota', 'Volkswagen', 'Volvo', 'n'],
      dtype='object')